# Agent Based Modeling: Virus on a Network
This model is based on the NetLogo model Virus on a Network (http://ccl.northwestern.edu/netlogo/models/VirusonaNetwork) and demonstrates the spread of a virus through a social network.  It consists of 3 compartments: $S$, the number of susceptible individuals, $I$, the number of infected individuals, and $R$, the number of recovered (and immune) individuals.  (An assumption is that individuals have life-long immunity, so this would not be the appropriate model for the flu or for COVID-19.)
The code below is explained in pieces.  It will need to be run in Netlogo to execute it (it can't be run from this notebook).  Stated alternatively, this Jupyter notebook is used for an explanantion of the code, but the code can be either run in NetLogo if that is installed locally on your machine, or can be run in the NetLogo webbrowser at this site: http://www.netlogoweb.org/launch#http://ccl.northwestern.edu/netlogo/models/models/Sample%20Models/Networks/Virus%20on%20a%20Network.nlogo.

### How it works
This model simulates:
- Agents on a network that can get infected and recover or become immune.
- Virus spreading through links (edges).
- Stochastic recovery and immunity behavior.
- Visual feedback using colors:
    - Red = infected
    - Blue = susceptible
    - Gray = resistant
    
The model defines several sliders that allow the user to customize some parameters in the model.    
    
At each time step (tick), each infected node (red) attempts to infect all of its neighbors.  Susceptible neighbors (blue) will be infected with a probability given by the VIRUS-SPREAD-CHANCE slider.  Resistant nodes (gray) can't be infected.

Infected nodes don't know they are infected-- the VIRUS-CHECK-FREQUENCY slider determines the frequency that nodes check to see if they are infected.  Once detected, the infection is cleared (recovered with a probability determined by the RECOVERY-CHANCE slider), and there is some probability that the node will become resistant (gray) to the virus, determined by the GAIN-RESISTANCE-CHANCE slider.

When a node becomes resistant, the links between it and its neighbors are darkened to indicate it can no longer spread the infection.

### Run the Model
Before looking at the code, lets look at a few runs of the model.
1. Set the sliders:

Choose the NUMBER-OF-NODES and the AVERAGE-NODE-DEGREE (average number of links coming out of each node).

The network that is created is based on the Euclidean distance between nodes. A node is randomly selected and connected to the nearest node that it is not already connected to. This process is repeated until the network has the correct number of links to give the specified average node degree.

The INITIAL-OUTBREAK-SIZE slider determines how many of the nodes will start the simulation infected with the virus.  (You will see them as red.)

The VIRUS-SPREAD-CHANCE, VIRUS-CHECK-FREQUENCY, RECOVERY-CHANCE, and GAIN-RESISTANCE-CHANCE sliders should be set, too.  VIRUS-SPREAD-CHANCE is a parameter related to infectiousness, and the RECOVERY-CHANCE is a parameter related to immunity (or immune/virus interaction).  The GAIN-RESISTANCE-CHANCE parameter relates to if an individual will become resistant.  In this model, it is life-long resistance, but of course, in reality, we often have waning immunity (which is why there are COVID_19 boosters), and this can vary by age and other characteristics.

2. Press SETUP to create the network. 

3. Press GO to run the model. The model will stop running once the virus has completely died out.  (You can see the time in terms of ticks at the top.)

The NETWORK STATUS plot shows the number of nodes in each state (S, I, R) over time.

### Coding Conventions
NetLogo has some coding conventions that differ from Python, R and Matlab.  It is not necessary for you to know them in detail.  I will mention them here for people who use other languages heavily.
   - comments are done with ;;
   - variable names that return true of false have a ? ending their name, e.g., `infected?`
   - NetLogo uses `ask` to give instructons to specific agents
   - NetLogo uses `breed` to define agent types
   - there is no `return` keyword.  If you want a reporter to give a value, you define it as a reporter procedure
   - multiple agents operate simulataneously.  In one tick (time unit), turtles, patches, and links can all run their code via `ask`.
It is also good to know that many researchers publish their models and those models can be built upon (reference appropriately).  I give links you might want to explore in the Reference section of this notebook. 
 

### Define custom variables that each agent has
These variables let each turtle store its infection status, immunity status, and a counter for performing periodic virus checks.

In [ ]:
turtles-own
[
  infected?           ;; true if the turtle is currently infected
  resistant?          ;; true if the turtle cannot be infected anymore (immune)
  virus-check-timer   ;; tracks how many ticks since the turtle last checked for recovery
]


### Main Setup Procedure
This initializes the simulation: clears everything, creates and connects the turtles, infects a few of them, and gets ready to run.

In [ ]:
to setup
  clear-all                            ;; clears the world
  setup-nodes                          ;; creates turtles and positions them
  setup-spatially-clustered-network   ;; builds network links between turtles
  ask n-of initial-outbreak-size turtles
    [ become-infected ]               ;; randomly infects some turtles
  ask links [ set color white ]       ;; makes links visible
  reset-ticks                          ;; resets simulation time to 0
end


### Set up the Nodes
Creates `number-of-nodes` turtles (agents), places them randomly (but not too close to the edge), and gives each one a randomized timer for its next virus check.  Note the use of the randomization of coordinates.

In [ ]:
to setup-nodes
  set-default-shape turtles "circle"
  create-turtles number-of-nodes
  [
    setxy (random-xcor * 0.95) (random-ycor * 0.95)
    become-susceptible                      ;; start uninfected and non-resistant
    set virus-check-timer random virus-check-frequency
  ]
end


### Set up the Spatially Clustered Network
Creates a spatially clustered network:
- Links nearby turtles until a target number of links is reached.
- `layout-spring` makes the network visually pleasing by spacing agents apart using spring forces.

In [ ]:
to setup-spatially-clustered-network
  let num-links (average-node-degree * number-of-nodes) / 2
  while [count links < num-links ]
  [
    ask one-of turtles
    [
      let choice (min-one-of (other turtles with [not link-neighbor? myself])
                   [distance myself])
      if choice != nobody [ create-link-with choice ]
    ]
  ]
  repeat 10
  [
    layout-spring turtles links 0.3 (world-width / (sqrt number-of-nodes)) 1
  ]
end


### Main Simulation Loop
This runs on every tick:
1. Updates each turtle’s virus timer.
2. Spreads the virus from infected turtles.
3. Allows infected turtles to recover or become resistant.
4. Increments the simulation clock (`tick`).

In [ ]:
to go
  if all? turtles [not infected?]
    [ stop ]  ;; end simulation if nobody is infected
  ask turtles
  [
    set virus-check-timer virus-check-timer + 1
    if virus-check-timer >= virus-check-frequency
      [ set virus-check-timer 0 ]
  ]
  spread-virus
  do-virus-checks
  tick
end


### Infection Procedures
We have three blocks which alter the boolean value of the variables infected? and resistant? and change colors and links to visualize how immunity and infections spread.
1. become-infected:
This marks the turtle as infected and shows it as red.
2. become-susecptible:
Returns turtle to susceptible (not immune, not infected).
3. become-resistant:
Marks turtle as immune and turns links gray to visually show immunity spread.

In [ ]:
to become-infected
  set infected? true
  set resistant? false
  set color red
end

to become-susceptible
  set infected? false
  set resistant? false
  set color blue
end

to become-resistant
  set infected? false
  set resistant? true
  set color gray
  ask my-links [ set color gray - 2 ]
end



### Virus Spread
1. We model the spread of the virus:
- Infected turtles try to infect their linked neighbors, with a chance based on `virus-spread-chance`.
2. We also do virus checks.  Each infected turtle, at a virus-check interval:
- May recover (`recovery-chance`)
- If it recovers, it may become resistant (`gain-resistance-chance`) or just susceptible again.

to spread-virus
  ask turtles with [infected?]
    [ ask link-neighbors with [not resistant?]
        [ if random-float 100 < virus-spread-chance
            [ become-infected ] ] ]
end


In [ ]:
to do-virus-checks
  ask turtles with [infected? and virus-check-timer = 0]
  [
    if random 100 < recovery-chance
    [
      ifelse random 100 < gain-resistance-chance
        [ become-resistant ]
        [ become-susceptible ]
    ]
  ]
end


### BehaviorSpace for Parameter Sweeps and Batch Experiments
It can be very useful to collect data from multiple runs of a NetLogo model and analyze or visualize the data using Python.  

NetLogo has a feature called BehaviorSpace that allows you to systematically vary parameters and record output data to study how different conditions affect the behavior of an agent-based model.  Its key features are 


- Batch Runs: Automates multiple runs of the model with different parameter settings.
- Parameter Sweeps: Vary one or more parameters across a range of values (e.g., test 5 different birth rates).
- Replications: Run the same set of parameters multiple times to account for stochasticity (randomness).
- Data Collection: Automatically collects values from reporters (like the number of infected) at each step or at the end of the run.
- Export Options: Results can be saved as CSV files for later analysis (e.g., in Excel, R, or Python).

This cannot be used from the NetLogo Web, but can be used in NetLogo if it is downloaded to your machine.  The dowload is free.

As an example, we could vary two parameters in the model by setting `initial-outbreak-size: 1 5 10` and
`chance-recovery: 0.01 0.05 0.1 0.2` which gives 12 combinations to test (3 choices for the first parameter and 4 choices for the second).  We can choose metrics to record ("reporters") such as  how many infected and how many recovered
- `count turtles with [infected?]`
- `count turtles with [recovered?]`

BehaviorSpace will save the results as a `.csv` file where each row corresponds to a run (we might do multiple runs to account for stochasticity in the model) and includes the parameters being used, the values of the reporters and the run number.  You can then use this data to create graphs, or calculate statistics.

There is also a Python package (pyNetLogo) that will let you control NetLogo from within Python, and can be used as an alternative to BehaviorSpace.  It will require you to have the model `.nlogo` file saved locally.  You can export a model from the NetLogo web by choosing the Export:NetLogo button (top right of screen).  This will save a `.nlogo` file, and you will need to note its path.  I am just learning about this, and find it a bit fussy, but feel free to explore it.

### Suggestions
Look at the NetLogo Modeling Commons and find two other models that relate to epidemics.  (You can search COVID-19 or infection or outbreak.)  
- How did modelers incorporate other assumptions into various models?  For example, do any models use quarantine, social distancing, masking, allow individuals to die, include the use of vaccination or boosters, or have different rules for different aged individuals? 
- Besides the assumptions and rules you have found looking through existing models, what else might you want to include in an epidemic model?  
- How is stochasticity incorporated in models you looked at?  Do you have any other ideas for how you might use this?
- How is heterogeneous space incorporated in the models you looked at?  Do you have any other ideas for how you might use this?

What are shortcomings of the Virus on a Network model?  Stated alternatively, what types of changes might you make so the model is improved?  (This is just a brainstorming question.)


### Local Expert on ABMs
Steve Railsback wrote a really wonderful book (cited below) about Agent-based models.  He is a world reknowned expert, and gives seminars internationally.  He is also an adjunct professor in our department and works on ecological modeling locally. I highly recommend his book if you want to delve into ABMs!
https://www.humboldt.edu/ecological-modeling/who-we-are
https://www.railsback-grimm-abm-book.com

### Butterfly Example
This example in Dr. Railsback's book is pretty cool and comes from a paper (Pe'Er, 2005 -- cited below) modeling butterfly movement and mating behavior on hilltops, important in conservation biology.  It incorporates rules for agent behavior, but also a makes use of an elevation profile (from a file).  This shows the power of ABM in that we can incorporate data from the environment in our model.


This is an ODD (Overview, Design concepts, Details) Description of the model: https://www.railsback-grimm-abm-book.com/E2-Downloads/Chapter04/ButterflyModelODD.txt.
ODD is a standard way to explain ABMs, and you will notice this in some of the Model Information in the NetLogo User Community.
More information abotu the ODD protocol can be found here: https://www.jasss.org/23/2/7.html.

We will work through this model.

### Code
I coded this in NetLogo 7 and will also put the file (extension is .nlogox for version 7 and beyond) on Canvas.  I will paste the code here, too. Again, you will have to run the model in Netlogo, not this Jupyter notebook (so copy and paste the code).
In NetLogo, comments are done with `;`.
It will be helpful to look at the ODD as you read the code.  For example, the initialization described in the ODD corresponds to the `setup` in the code.

This NetLogo model sets up a simple world where butterflies move around a hilly landscape defined by patch elevations. The variable `q` is declared as a global, which means it has the same value for all turtles and patches. In this case, `q` represents the probability that a butterfly will move directly to the highest surrounding patch rather than wandering randomly. Each patch has its own state variable `elevation`, which is calculated from a sine function of its `(pxcor, pycor)` coordinates. This function produces a patterned landscape with repeating hills and valleys, and the elevation is mapped to patch color using a green scale so you can visually track the terrain. During `setup`, the model clears everything, assigns elevations and colors to patches, creates 500 butterflies at random starting positions, and sets `q` to 0.4 (meaning butterflies will follow the “go uphill” rule 40% of the time).


Once `setup` is complete, the scheduler (`go`) drives the simulation. Each tick represents one unit of time, and at each tick every butterfly executes the `move` procedure. The movement decision uses `ifelse random-float 1.0 < q`. This condition is true with probability q and false with probability 1 – q. When true, the butterfly uses the built-in `uphill` primitive to move toward the neighbor patch with the highest elevation. Otherwise, the butterfly chooses a random neighbor with `move-to one-of neighbors`. Because of this probabilistic rule, the butterflies sometimes act purposefully by climbing hills and other times wander randomly. The simulation continues until 1000 ticks have passed, at which point it stops automatically. Overall, the model illustrates how simple rules combining probability and the environment can produce diverse and interpretable movement patterns.

In NetLogo, you can create a new file, save it as Butterfly-1.nlogox.  At the top of the interface, you will see a way to toggle between `Interface`, `Info` and `Code` buttons.  You can paste the ODD text into the Info screen and the code into thte Code screen.

In [ ]:
globals [ q ] ; q is the probability that butterfly moves directly to highest surrounding patch

; state variables
patches-own [ elevation ] ; patches have an elevation state variable
turtles-own [ ] ; by default, the location of the turtles is already tracked as (xcor, ycor) so we don't have to list it

to setup
  ca ; stands for clear all, usually the first line in the setup procedure
  
  ; Assign an elevation to patches and color them by it 
  ask patches
  [ 
    ; Elevation is a sine function of X,Y coordinates-- this is just a way to set up a nice test situation with four hilltops for the size grid we use
    ; this function has a max at elevation 400 and a min at 0
    set elevation 200 + (100 * (sin (pxcor * 3.8) + sin (pycor * 3.8)))
    
    set pcolor scale-color green elevation 0 400 ; sets a color scale for elevation to help us visualize hills
  ] ; end of "ask patches"
  
  
  ; Create Butterflies
  crt 500
  [ 
    set size 2 ; this is just visual so they are easier to see
    ; set initial location to random patch
    setxy random-pxcor random-pycor
    pen-down ; this lets us see their paths
  ]
  
  ; Initialize q parameter
  set q 0.4 ; 
  
  
  reset-ticks ; like resetting a stopwatch at the start of an experiment, almost always the last line in the setup procedure
end; end of setup procedure

to go ; this is the scheduler
  ask turtles [move] ; we will define this procedure below
  tick
  if ticks >= 1000 [stop] ; end time is set to be 1000, when reached, stop
end

to move ; the butterfly move procedure idecides whether to move to the highest surrounding patch with probability q, otherwise selects a random neighbor
  ifelse random-float 1.0 < q ; random-float 1.0 generates a random decimal number between 0 (inclusive) and 1 (exclusive). 
 ; With probability q the first block of code runs, with probability 1-q, the second block runs
  [ uphill elevation] ; move deterministically uphill
  [move-to one-of neighbors ] ; or move randomly to a neighbor
end ; ends move procedure

### Graphical User Interface
There are a few things we need to do with the GUI to be set.  
- On the Interface option, click `Settings`.  For location of origin, choose corner and bottom left.  For max-pxcor and max-pycor, enter 149 (this will give us a grid of 150 x 150 patches).  Choose 3 for patch size (helps us view them.)  Cick apply.
- On the Interface option, and add a widget (button) with the command `setup`.  This links the `setup` procedure you wrote in your code to this button.  When clicked, it sets up the model with the proper initialization.
- Add another button and enter `go` for the commands.  This links the button to the go procedure.  Click the `forever` box.  This means you can click the go button once and it will run until end time, instead of having to click for each tick.
- Save everything.
- You are ready to run your model.  Click setup.  You should see a topography with hills and butterflies initialized in random locations.  Click go.  You should see paths from the 500 butterflies.

### Try:
- Try some different values for the parameter $q$.  Do you notice any trends?
- Try adding noise to the landscape.  You could add this statement to the `setup` just after the patch elevation is set: `set elevation elevation + random-float 20.0`.

### Moving Beyond Visual Output
This model creates a lovely animation, and gives us some intuition about the behavior of butterflies in this topography.  For the model to serve its scientific purpuse (understanding the emergence of virtual corridors) we need additional outputs to quantify the width of the corridors.  We also might be interested in a realistic landscape instead of an artificial landscape we create.  So instead of using the model as a simulator, we need to produce quantitative output that can be used to answer questions.
For the next version of the model, we will
- define the corridor width by (a) the number of patches that are visited by any butterflies divided by (b) the mean distance between starting and ending locations, over all butterflies.  This means we need a way to determine if a patch has been visited by any butterfly, and need a way to know where each butterfly starts and ends.  Details on this bulleted below.
- vary $q$ and see how this changes the corridor width.  To do this, using the interface tab, add a slider (under widgets) for $q$.  Let $q$ vary in $[0,1]$ with step size $.01$.  When you do this, your code will give you a warning-- you need to remove $q$ from the global variables in the code.  We also need to remove the line in `setup` that initializes $q$.  (If left, the code will use this value instead of the value with the slider.)  I commented these lines out so you can see the change.
- add a new variable named `used?` to the patches-own statement.  This is a boolean variable (true or false) and NetLogo has a convention of ending boolean variables with a question mark.  This variable will be true if any butterfly has landed on the patch.
- add a variable caleld `start-patch` to the `turtles-own` statement.
- in the `set-up` procedure, add statements to initialize these new variables.  At the end of the `ask patches` you will add `set used? false` and at the end of the create butterflies section, set `start-patch` to the patch on which the butterfly is located.  There is a built in NetLogo function `patch-here` that reports the patch a turtle is currently on.
- add a statement to the `move` procedure that causes the butterfly, once it has moved to a new patch, to set the patch's variable `used?` to true. 


### Calculate corridor width
At the end of the simulation, we need to calculate the corridor width.  
We will do this using a reporter procedure, which is something that reports a value.  NetLogo has some build in reporters like `count turtles` (how many exists), and also allows us to define them.  When defined, they begin with `to-report` and end with `end`.  Inside you compute something and finish with a `report` statement.  (For programmers in other languages, this probably reminds you of a function and the keyword `return`.)
- Let's report the number of used patches
- Let's report the mean start to end distance of butterflies
- Let's put these together into a quotient to report the corridor-width.
Let's also add an output element to the interface.
- in the `go` procedure, we will add a statement to print out the coordior width
- in the Interface table, add an Output widget
You will see this coded at the bottom of the updated version:

In [ ]:
;globals [ q ] ; q is the probability that butterfly moves directly to highest surrounding patch, in the second iteration we added a slider for q so removed this

; state variables
patches-own [ elevation used? ] ; patches have an elevation state variable, in the second iteration, we added a new variable used?
turtles-own [  start-patch ] ; changed in version 2-- we want to track where each butterfly starts to calculate their net distance later

to setup
  ca ; stands for clear all, usually the first line in the setup procedure
  
  ; Assign an elevation to patches and color them by it 
  ask patches
  [ 
    ; Elevation is a sine function of X,Y coordinates-- this is just a way to set up a nice test situation with four hilltops for the size grid we use
    ; this function has a max at elevation 400 and a min at 0
    set elevation 200 + (100 * (sin (pxcor * 3.8) + sin (pycor * 3.8)))
    set pcolor scale-color green elevation 0 400 ; sets a color scale for elevation to help us visualize hills
    set used? false ; initialize the used? variable.  at the beginning, no visits have occured
  ] ; end of "ask patches"
  
  
  ; Create Butterflies
  crt 500
  [ 
    set size 2 ; this is just visual so they are easier to see
    ; set initial location to random patch
    setxy random-pxcor random-pycor
    pen-down ; this lets us see their paths
    set start-patch patch-here ; updated for this version, remember starting patch
  ]
  
  ; Initialize q parameter
  ;set q 0.4 ; in the second iteration, we added a slider for q to the interface so removed this
  
  
  reset-ticks ; like resetting a stopwatch at the start of an experiment, almost always the last line in the setup procedure
end; end of setup procedure

to go ; this is the scheduler
  ask turtles [move] ; we will define this procedure below
  tick
  if ticks >= 1000 [
    output-print (word "Corridor width = "  corridor-width) ; this is updated in version 2 so that the corridor width is output to the interface
    stop] ; end time is set to be 1000, when reached, stop
end

to move ; the butterfly move procedure idecides whether to move to the highest surrounding patch with probability q, otherwise selects a random neighbor
  ifelse random-float 1.0 < q ; random-float 1.0 generates a random decimal number between 0 (inclusive) and 1 (exclusive). 
 ; With probability q the first block of code runs, with probability 1-q, the second block runs
  [ uphill elevation ] ; move deterministically uphill
  [ move-to one-of neighbors ] ; or move randomly to a neighbor
  ask patch-here [ set used? true] ; added om version 2 to track if a patch has ever been visited by a butterfly
end ; ends move procedure

;;;;;; ADD REPORTERS, addition in version 2
; counts how many patches have their used? value set to true, meaning at least one butterfly visited the patch
to-report used-patches-count
  report count patches with [ used? ]
end

; calculates mean of all butterflies' net distance
to-report mean-start-end-distance
  let dists [ distance start-patch ] of turtles ; dists is a list of net distance for each butterfly
  report mean dists ; average and report
end

; calculates corridor width as defined.  includes a guard against division by zero
to-report corridor-width
  let denom mean-start-end-distance
  if denom = 0 [ report 0 ] ; guard against dividing by zerp
  report used-patches-count / denom
end


### Suggestions:
- Run this in NetLogo.  Run it for a very low $q$ (choose a value with the slider), a medium $q$ and a high $q$.  What do you notice visually?  How does corridor width change?
- NetLogo has a cool feature called Behavior Space which is like a build in experiemnt manager.  Instead of manually changing sliders, running the model, recording results, it can do it automatically for you.  It collects results in a CSV file for later analysis.  Google this to get an idea of how it works.  This can be quite powerful if you have many parameters.  Varying parameters like this is called a parameter sweep and this can be helpful to understand the sensititvity (does a small change in a parameter cause a big change in output) and to to discover thresholds (akin to bifurcations for ODEs).

### References and Resources

#### Papers
Stonedahl, F. and Wilensky, U. (2008). NetLogo Virus on a Network model. Center for Connected Learning and Computer-Based Modeling, Northwestern University, Evanston, IL.

Wilensky, U. (1999). NetLogo Center for Connected Learning and Computer-Based Modeling, Northwestern University, Evanston, IL.

Railsback, Steven F., and Volker Grimm. Agent-based and individual-based modeling: a practical introduction. Princeton university press, 2019.

Pe'Er, G. U. Y., David Saltz, and Karin Frank. "Virtual corridors for conservation management." Conservation biology 19.6 (2005): 1997-2003.  PDF is here: https://ees4760.jgilligan.org/files/reading/Peer-VirtualCorridors-2005.pdf

#### Useful Links
NetLogo User Community Models (location where researchers upload and explain their models): http://ccl.northwestern.edu/netlogo/models/community/index.cgi

Fun fact: My student researchers have 2 models published on this site!  These relate to a type of immune cell in the brain.
- http://ccl.northwestern.edu/netlogo/models/community/Microglia%20Model
- http://ccl.northwestern.edu/netlogo/models/community/index.cgi

NetLogo Modeling Commons is a site for discussion and sharing agent-based models written in NetLogo.  http://modelingcommons.org/account/login